# <p><center style="font-family:newtimeroman;font-size:180%;">Alzheimers Dataset 4 Class ADNI Detection and Quality Evaluation Using Image Analysis </center></p>
### Table of contents:

* [Import Libraries](#1)
* [Read Data](#2)
* [Visualization Of Dataset](#3)
* [Data Augmentation](#4)
* [InceptionV3 Model](#10)
* [Evaluate The Model](#11)


**<a id="1"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Import Libraries </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
!pip install seaborn

In [1]:
import os 
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
import keras 
from keras.callbacks import EarlyStopping,ModelCheckpoint
import tensorflow as tf
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tqdm import tqdm
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import tensorflow.compat.v1 as tf
import skimage
import skimage.io

**<a id="2"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Read Data </p>
<a class="btn" href="#home">Tabel of Contents</a>

**<a id="3"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">Visualization of Dataset </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
Current_path='/kaggle/working'
sub2=[]
lastfolder=[]
MainParent=os.path.join(Current_path,'AlzheimerDataset')
for subfolder1 in tqdm(os.listdir('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset')):
    subfolder1 = os.path.join('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset', subfolder1)
    subfolder2=os.path.join(MainParent,os.path.basename(subfolder1))
    if not os.path.exists(subfolder2):
            os.makedirs(subfolder2)
    sub2.append(subfolder2)
    for folder in np.sort(os.listdir(subfolder1)):
        last=os.path.join(subfolder2,folder)
        if not os.path.exists(last):
            os.makedirs(last)
        lastfolder.append(last)

In [ ]:
sub2,lastfolder

In [ ]:
from PIL import Image
import os

for subfolder in tqdm(os.listdir('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset')):
    subfolder_path = os.path.join('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset', subfolder)
    for i in range(len(sub2)):
        if subfolder==os.path.basename(sub2[i]):
            for j in range(len(lastfolder)):
                for folder in os.listdir(subfolder_path):
                    subfolder_path2=os.path.join(subfolder_path,folder)
                    if os.path.basename(subfolder_path2)==os.path.basename(lastfolder[j]):
                        for image_filename in os.listdir(subfolder_path2):
                            if image_filename.endswith(".jpg") or image_filename.endswith(".png") or image_filename.endswith(".JPG"):
                                #if sub2[i]==0:
                                    #image=extract_image_index_make_label_1(image_filename)
                                    #print(type(image))
                                image = Image.open(os.path.join(subfolder_path2,image_filename))
                                #else:
                                  #  image=extract_image_index_make_label_0(image_filename)
                                  #  print(type(image))
                                #image = Image.open(image) 
                                #print(image)
                                # Resize the image to 512x512
                                resized_image = image.resize((512, 512))
                                image_rgb = resized_image.convert("RGB")
                                #os.path.join(lastfolder[j],image_filename)
                                #print(image_rgb)
                                #print(lastfolder[j])
                                image_rgb.save(os.path.join(lastfolder[j],image_filename))

In [ ]:
!ls /kaggle/working

In [ ]:
from PIL import Image
import pathlib
from pathlib import Path
import os

images = []
labels = []
for subfolder in tqdm(os.listdir('/kaggle/working/AlzheimerDataset')):
    subfolder_path = os.path.join('/kaggle/working/AlzheimerDataset', subfolder)
    for folder in os.listdir(subfolder_path):
        subfolder_path2=os.path.join(subfolder_path,folder)
        for image_filename in os.listdir(subfolder_path2):
           # if image_filename.endswith(".jpg") or image_filename.endswith(".png") or image_filename.endswith(".JPG"):
            image_path = os.path.join(subfolder_path2, image_filename)
            #image_path = os.path.join(source_dir, filename)
            images.append(image_path)
            labels.append(folder)
df = pd.DataFrame({'image': images, 'label': labels})

<a id="4"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px"> Data Augmentation </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
plt.figure(figsize=(15,8))
ax = sns.countplot(x=df.label,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
Size=(176,176)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
train_data_gen = work_dr.flow_from_dataframe(df,x_col='image',y_col='label', target_size=Size, batch_size=6500, shuffle=False)

In [ ]:
train_data, train_labels = train_data_gen.next()

In [ ]:
sm = SMOTE(random_state=42)
train_data, train_labels = sm.fit_resample(train_data.reshape(-1, 176 * 176 * 3), train_labels)
train_data = train_data.reshape(-1, 176,176, 3)
print(train_data.shape, train_labels.shape)

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(df),50)):
    plt.subplot(10,5,n+1)
    img=cv2.imread(df.image[i])
    img=cv2.resize(img,(224,224))
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.axis('off')
    plt.title(df.label[i],fontsize=25)

In [ ]:
class_num=np.sort(['MildDemented','ModerateDemented','NonDemented','VeryMildDemented'])
class_num

In [ ]:
labels=[class_num[i] for i in np.argmax(train_labels,axis=1) ]
plt.figure(figsize=(15,8))
ax = sns.countplot(x=labels,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

In [ ]:
X_train, X_test1, y_train, y_test1 = train_test_split(train_data,train_labels, test_size=0.3, random_state=42,shuffle=True,stratify=train_labels)
X_val, X_test, y_val, y_test = train_test_split(X_test1,y_test1, test_size=0.5, random_state=42,shuffle=True,stratify=y_test1)
print('X_train shape is ' , X_train.shape)
print('X_test shape is ' , X_test.shape)
print('X_val shape is ' , X_val.shape)
print('y_train shape is ' , y_train.shape)
print('y_test shape is ' , y_test.shape)
print('y_val shape is ' , y_val.shape)

<a id="10"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px">InceptionV3 Modeling </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
def read_image(file_name, images_path):
    image = skimage.io.imread(images_path + file_name)
    return image

In [ ]:
# Source: https://stackoverflow.com/questions/33849617/how-do-i-convert-a-directory-of-jpeg-images-to-tfrecords-file-in-tensorflow
# Note: modified from source
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# images and labels array as input
def convert_to(images, labels, output_directory, name):
    num_examples = labels.shape[0]
    if images.shape[0] != num_examples:
        raise ValueError("Images size %d does not match label size %d." %
                         (images.shape[0], num_examples))
    rows = images.shape[1]
    cols = images.shape[2]
    depth = 1

    filename = os.path.join(output_directory, name + '.tfrecords')
    print('Writing', filename)
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(num_examples):
        image_raw = images[index].tobytes()
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(rows),
            'width': _int64_feature(cols),
            'depth': _int64_feature(depth),
            'label': _int64_feature(int(labels[index])),
            'image_raw': _bytes_feature(image_raw)}))
        writer.write(example.SerializeToString())

In [ ]:
import re
from functools import reduce  
res=('moderateDem', '40')
pre=reduce(lambda x,y:x+y,[ord(i) for i in res[0][0]])
sufex=res[1]
pre,sufex
#ord('moderateDem')
type(res[0])
s = 'hi'
res[0][:4]
[ord(c) for c in s]
x='%d'*len(res[0][:4]) % tuple(map(ord, res[0][:4]))
int(x)
sufex

In [ ]:
def extract_image_index_make_label_1(img_name):
    import re
    from functools import reduce
    remove_ext = img_name.split(".")[0]
    res = re.findall(r'(\w+?)(\d+)', remove_ext) 
    x=list(sum(res, ()))
    label=reduce(lambda x, y: str(x) + str(y),x)
    #res = [re.findall(r'(\w+?)(\d+)', remove_ext)[0] ]
    #prefex=reduce(lambda x,y:x+y,[ord(i) for i in res[0][0]])
    #sufex=res[0][1]
    #name, serie, repetition, char = remove_ext.split("_")
    #label=int(str(sufex)+str(prefex))
    #label = int(char) + 1000 * int(repetition) + 1000_000 * int(serie)
    #print(int(label))
    return  int(label)#remove_ext

In [ ]:
def extract_image_index_make_label_0(img_name):
    import re
    from functools import reduce
    #print(len(img_name))
    remove_ext = img_name.split(".")[0]
    #res = re.findall(r'(\w+?)(\d+)', remove_ext) 
    #x=list(sum(res, ()))
    #label=reduce(lambda x, y: str(x) + str(y),x)
    #print(remove_ext)
    res = [re.findall(r'(\w+?)(\d+)', remove_ext)[0] ][0]
    #print(res)
    prefex=int('%d'*len(res[0][:4]) % tuple(map(ord, res[0][:4])))
    #prefex=reduce(lambda x,y:x+y,[ord(i) for i in res[0][0]])
    sufex=int(res[1])
    #name, serie, repetition, char = remove_ext.split("_")
    #print(sufex),print(prefex)
    label=int(str(sufex)+str(prefex))
    #label = int(char) + 1000 * int(repetition) + 1000_000 * int(serie)
    #print(type(label))
    return  label#remove_ext

In [ ]:
import re
s="26 (44)"
pattern = r'\(.*?\)'
#res=re.findall(r'\[[^()]*\]', s)
#res = re.sub(pattern, "0", s)
#s = "start (inside) this is in between (inside) end"
res = re.findall(r'(\w+?)(\d+)',s) 
#res = re.sub(r"\((\d+)\)", r"00", s)

#list(sum(output, ()))
#res = re.sub(r"\((.*?)\)", "00", s)
print(res) 
from functools import reduce
type(res[0])
x=list(sum(res, ()))
z=reduce(lambda x, y: str(x) + str(y),x)
int(z)

In [ ]:
import os
import glob

original_path = '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/'

def get_subfolders(path):
    return [f.path for f in os.scandir(path) if f.is_dir()]

def get_files_in_subfolder(subfolder, extension):
    return list(glob.glob(subfolder + '/*' + extension))

Reultsfiles = []
subfolders = [original_path] + get_subfolders(original_path)
files=[]
directory=[]
while len(subfolders) > len(get_subfolders(original_path))-1:
    new_subfolder = subfolders.pop()
    #folders.append(new_subfolder)
    if len(subfolders)>0:
        subfolders += get_subfolders(new_subfolder)
        directory.append(new_subfolder)
        files += get_files_in_subfolder(new_subfolder, '.jpg')
        Reultsfiles.append(files)

In [ ]:
def PathLength(path: str):
      path = path.split('/')
      spaces = path.count('')
      for s in range(spaces):
            path.remove('')
      return len(path)
LongPath=PathLength(directory[0])
for i,x in enumerate(directory):
    if PathLength(directory[i])<LongPath:
        LongPath=PathLength(directory[i])

In [ ]:
NewDir=[]
Perents=[]
#Dir=None
for i in range(len(directory)):
    if PathLength(directory[i])!=LongPath:
        #Dir= directory[i].pop()
        NewDir.append(directory[i])
    else:
       Perents.append(directory[i])
        
#len(NewDir) ,len(directory) ,Perents      

In [ ]:
Perents

In [ ]:
images = []
labels = []
for subfolder in tqdm(os.listdir("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset")):
    subfolder_path = os.path.join("/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset", subfolder)
    print(subfolder_path)
    break
    for folder in os.listdir(subfolder_path):
        subfolder_path2=os.path.join(subfolder_path,folder)
        for image_filename in os.listdir(subfolder_path2):
            image_path = os.path.join(subfolder_path2, image_filename)
            images.append(image_path)
            labels.append(folder)
df = pd.DataFrame({'image': images, 'label': labels})

In [ ]:
#'MildDemented', 'ModerateDemented', 'NonDemented',  'VeryMildDemented'
images_path = "/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/VeryMildDemented/"
image_list = os.listdir(images_path)
images = []
labels = []
for img_name in tqdm(image_list):
    images.append(read_image(img_name, images_path))
    labels.append(extract_image_index_make_label_0(img_name))
    images_array = np.array(images)
labels = np.array(labels)
print(images_array.shape, labels.shape)
#print(type(labels))

In [ ]:
images_array.shape[0]

In [ ]:
from pathlib import Path
images = []
labels = []
lists=[]
finall=[]
labels = np.array([]) 
for i in range(len(NewDir)):
    path=Path(NewDir[i])
    if path.parent.absolute()==Path(Perents[0]):
        newpath=os.path.join(Path(NewDir[i]),'')
        lists.append(newpath)
        image_list = os.listdir(str(lists[i]))
        for img_name in tqdm(image_list):
            images.append(read_image(img_name, lists[i]))
            labels=np.append(labels,np.array([extract_image_index_make_label_0(img_name)]))
    images_array = np.array(images)
    labels = np.array(labels)
    finall.append([images_array,labels])
    images.clear()
    labels=np.array([])
    if path.parent.absolute()==Path(Perents[1]):
        newpath=os.path.join(Path(NewDir[i]),'')
        lists.append(newpath)
        image_list = os.listdir(str(lists[i]))
        for img_name in tqdm(image_list):
            images.append(read_image(img_name, lists[i]))
            labels=np.append(labels,np.array([extract_image_index_make_label_1(img_name)]))
    images_array = np.array(images)
    labels = np.array(labels)
    finall.append([images_array,labels])
    images.clear()
    labels=np.array([])

In [ ]:
len(finall[0][0]),
len(finall[0][0][0][0])
finall[0][0].shape
finall[0][1].shape

In [ ]:
len(finall[0])
len(fin[0])

In [ ]:
fin=[]
for i in range(len(finall)):
    if finall[i][0].shape[0]!=0:
        fin.append([finall[i][0],finall[i][1]])

In [ ]:
newpath=[]
subfolder=[]
for i in range(len(Perents)):
    subfolder.append(os.path.basename(Perents[i]))
for i in range(len(subfolder)):
    newpath.append(os.path.join('/kaggle/working/',subfolder[i]) )
    if not os.path.exists(newpath[i]):
        os.makedirs(newpath[i])
newpath

In [ ]:
#a="/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset"
#sub = [ f.path for f in os.scandir(a) if f.is_dir() ]
finalPath=[]
for i in range(len(Perents)):
    for k in tqdm(os.listdir(Perents[i])):
             if  subfolder[i]==os.path.basename(newpath[i]) and os.path.basename(Perents[i])==subfolder[i]:
                if not os.path.exists(os.path.join(newpath[i],k)):
                        os.makedirs(os.path.join(newpath[i],k))
                        finalPath.append(os.path.join(newpath[i],k))
                else:    
                        finalPath.append(os.path.join(newpath[i],k))

In [ ]:
finalPath

In [ ]:
for i in range(len(fin)):
    convert_to(fin[i][0], fin[i][1], finalPath[i],os.path.basename(finalPath[i]))

In [ ]:
for subfolder in tqdm(os.listdir('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset')):
    subfolder_path = os.path.join('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset', subfolder)
    for folder in os.listdir(subfolder_path):
        subfolder_path2=os.path.join(subfolder_path,folder)
        for image_filename in os.listdir(subfolder_path2):
            if image_filename.endswith(".jpg") or image_filename.endswith(".png") or image_filename.endswith(".JPG"):
                image = Image.open(image_path)
                # Resize the image to 512x512
                resized_image = image.resize((512, 512))
                image_rgb = resized_image.convert("RGB")
                # check if file exist or not
                #paths=os.path.dirname(image_path)
                new_image_path = os.path.join(paths, image_filename)
                image_rgb.save(output_path)
                


In [ ]:
# add align_images.py
!git clone https://github.com/rkuo2000/stylegan2-ada-pytorch
%cd stylegan2-ada-pytorch

In [ ]:
!pip install click requests tqdm pyspng ninja imageio-ffmpeg==0.4.3

In [ ]:
import os
os.mkdir("/kaggle/working/my_new_dir")

In [ ]:
!python /kaggle/working/stylegan2-ada-pytorch/dataset_tool.py --source='/kaggle/working/train/MildDemented' --dest=/kaggle/working/my_new_dir

In [ ]:
!python /kaggle/working/stylegan2-ada-pytorch/dataset_tool.py --source=/kaggle/input/adni-screening-1-5t-segmented-complete-dataset/ADNI-Screening-1.5T-Segmented-Complete-Dataset/Extracted_AD --dest=/kaggle/working/my_new_dir/cc.zip


In [ ]:
#python train.py --outdir=~/training-runs --data=~/mydataset.zip --gpus=1 --dry-run
!python train.py --outdir=/kaggle/working/my_new_dir/training-runs --data='/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented' --gpus=1 --cfg=paper512 --aug=ada --target=0.7 

In [ ]:
! git clone https://github.com/mckellwoodland/fid-med-eval.git  /kaggle/working/ss

In [ ]:
! python /kaggle/working/ss/utils/prepare_MSDbt.py  --in_dir /kaggle/working/train/ModerateDemented --out_dir /kaggle/working/output


In [ ]:
!git clone  https://github.com/sh4174/3DStyleGAN.git /kaggle/working/3DStyleGAN

In [ ]:
!python /kaggle/working/3DStyleGAN/dataset_tool.py create_from_images3d  /kaggle/working/my_new_dir   '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented'  --shuffle 1 --base_size 5 6 7 #--image_dir '/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset/train/MildDemented'

In [ ]:
python run_training.py --num-gpus=4 --data-dir=/kaggle/working/train/ModerateDemented --config=config-f --dataset=ModerateDemented.tfrecords --total-kimg=6000

In [ ]:
#%%capture
#%%bash
# clone repo
!git clone https://github.com/autonomousvision/projected_gan
#!git clone https://github.com/sarah127/stylegan-xl.git    
!pip install timm dill

In [ ]:
%%bash
python /kaggle/working/projected_gan/dataset_tool.py --source=/kaggle/input/afhq-512/cat --dest=/kaggle/working/datasets/cat.zip # --resolution=256x256


In [ ]:
fin[0][0][0],
len(fin[0][0])
fin[0][0][0]

In [ ]:
import numpy as np 
from PIL import Image as im 
len(fin[0][0][0])
img=[]
os.makedirs('/kaggle/working/try0')
#for folder in os.listdir(subfolder_path):
folder='/kaggle/working/try0'
for i in range(len(fin[0][0])):
    img.append(im.fromarray(fin[0][0][i]))    
    # saving the final output  
    # as a PNG file 
    img[i].save(f"{folder}/mildDem {str(i)}.png")
    #os.path.join('/kaggle/working/try',img[i] )


In [ ]:
import numpy as np 
from PIL import Image as im 
len(fin[1][0][0])
img=[]
os.makedirs('/kaggle/working/try1')
#for folder in os.listdir(subfolder_path):
folder='/kaggle/working/try1'
for i in range(len(fin[1][0])):
    img.append(im.fromarray(fin[1][0][i]))    
    # saving the final output  
    # as a PNG file 
    img[i].save(f"{folder}/ModDem {str(i)}.png")
    #os.path.join('/kaggle/working/try',img[i] )


In [ ]:
!ls '/kaggle/working/try'

In [ ]:
python /kaggle/working/projected_gan/dataset_tool.py --source='/kaggle/working/AlzheimerDataset/train/MildDemented' --dest=/kaggle/working/datasets/img1.zip

In [ ]:
%%bash
python /kaggle/working/projected_gan/dataset_tool.py --source='/kaggle/working/AlzheimerDataset/train/MildDemented' --dest=/kaggle/working/datasets/img0.zip
#--transform=center-crop-tall  --width=176 --height=208  #--resolution=256x256

In [ ]:
%%bash
python /kaggle/working/projected_gan/dataset_tool.py --source='/kaggle/working/AlzheimerDataset/train/ModerateDemented' --dest=/kaggle/working/datasets/img1.zip
#--transform=center-crop-tall  --width=176 --height=208  #--resolution=256x256

In [ ]:
%%bash
python /kaggle/working/projected_gan/dataset_tool.py --source='/kaggle/working/AlzheimerDataset' --dest=/kaggle/working/datasets/img.zip
#--transform=center-crop-tall  --width=176 --height=208  #--resolution=256x256

In [ ]:
import torch
torch.cuda.is_available()
cuda_device = torch.device("cuda")  # device object representing GPU

In [ ]:
import os
os.environ.get('CUDA_LAUNCH_BLOCKING', '1')
#os.environ['CUDA_LAUNCH_BLOCKING']= "1"
%env CUDA_LAUNCH_BLOCKING=$1

In [ ]:
#!pip install getpass
import getpass
import os


def SetEnv(name):
    #secret = getpass.getpass(f"Enter value of {name}: ")
    os.environ[name] = '1'

# to get var
#mys = os.environ['secret']
#print(mys)

# Usage to set var
SetEnv("CUDA_LAUNCH_BLOCKING")


In [ ]:
import sys

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

In [ ]:
%%capture
!pip install timm==0.5.4
!pip install ftfy
!pip install Ninja
!pip install setuptools==59.5.0
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
%cd /kaggle/working/projected_gan

In [ ]:
import os
import json
import re
import dnnlib

from training import training_loop
from torch_utils import training_stats
from train import init_dataset_kwargs
from metrics import metric_main

In [ ]:
def launch_training(c, desc, outdir, rank=0):
    # Pick output directory.
    prev_run_dirs = []
    if os.path.isdir(outdir):
        prev_run_dirs = [x for x in os.listdir(outdir) if os.path.isdir(os.path.join(outdir, x))]

    matching_dirs = [re.fullmatch(r'\d{5}' + f'-{desc}', x) for x in prev_run_dirs if re.fullmatch(r'\d{5}' + f'-{desc}', x) is not None]
    if c.restart_every > 0 and len(matching_dirs) > 0:  # expect unique desc, continue in this directory
        assert len(matching_dirs) == 1, f'Multiple directories found for resuming: {matching_dirs}'
        c.run_dir = os.path.join(outdir, matching_dirs[0].group())
    else:                     # fallback to standard
        prev_run_ids = [re.match(r'^\d+', x) for x in prev_run_dirs]
        prev_run_ids = [int(x.group()) for x in prev_run_ids if x is not None]
        cur_run_id = max(prev_run_ids, default=-1) + 1
        c.run_dir = os.path.join(outdir, f'{cur_run_id:05d}-{desc}')
        assert not os.path.exists(c.run_dir)


    # Print options.
    print()
    print('Training options:')
    print(json.dumps(c, indent=2))
    print()
    print(f'Output directory:    {c.run_dir}')
    print(f'Number of GPUs:      {c.num_gpus}')
    print(f'Batch size:          {c.batch_size} images')
    print(f'Training duration:   {c.total_kimg} kimg')
    print(f'Dataset path:        {c.training_set_kwargs.path}')
    print(f'Dataset size:        {c.training_set_kwargs.max_size} images')
    print(f'Dataset resolution:  {c.training_set_kwargs.resolution}')
    print(f'Dataset labels:      {c.training_set_kwargs.use_labels}')
    print(f'Dataset x-flips:     {c.training_set_kwargs.xflip}')
    print()

    # Create output directory.
    print('Creating output directory...')
    os.makedirs(c.run_dir, exist_ok=c.restart_every > 0)
    with open(os.path.join(c.run_dir, 'training_options.json'), 'wt+') as f:
        json.dump(c, f, indent=2)

    # Start training
    dnnlib.util.Logger(file_name=os.path.join(c.run_dir, 'log.txt'), file_mode='a', should_flush=False)
    sync_device = torch.device('cuda', rank) if c.num_gpus > 1 else None
    training_stats.init_multiprocessing(rank=rank, sync_device=sync_device)
    training_loop.training_loop(rank=rank, **c)

In [ ]:
!unzip 

In [ ]:
def train(**kwargs):
    # Initialize config.
    #from stylegan2 
    import dnnlib
    import json
    import os
    import click
    import re
    import tempfile
    import torch
    import legacy
    from projected_gan import  pg_modules
    from training import training_loop
    from torch_utils import training_stats
    from train import init_dataset_kwargs
    from metrics import metric_main
    #from stylegan2.dnnlib.tflib.tfutil import * 
    opts = dnnlib.EasyDict(kwargs) # Command line arguments.
    c = dnnlib.EasyDict() # Main config dict.
    c.G_kwargs = dnnlib.EasyDict(class_name=None, z_dim=64, w_dim=128, mapping_kwargs=dnnlib.EasyDict())
    c.G_opt_kwargs = dnnlib.EasyDict(class_name='torch.optim.Adam', betas=[0,0.99], eps=1e-8)
    c.D_opt_kwargs = dnnlib.EasyDict(class_name='torch.optim.Adam', betas=[0,0.99], eps=1e-8)
    c.data_loader_kwargs = dnnlib.EasyDict(pin_memory=True, prefetch_factor=None)

    # Training set.
    c.training_set_kwargs, dataset_name = init_dataset_kwargs(data=opts.data)
    if opts.cond and not c.training_set_kwargs.use_labels:
        raise ValueError('--cond=True requires labels specified in dataset.json')
    c.training_set_kwargs.use_labels = opts.cond
    c.training_set_kwargs.xflip = opts.mirror

    # Hyperparameters & settings.
    c.num_gpus = opts.gpus
    c.batch_size = opts.batch
    c.batch_gpu = opts.batch_gpu or opts.batch // opts.gpus
    c.G_kwargs.channel_base = opts.cbase
    c.G_kwargs.channel_max = opts.cmax
    c.G_kwargs.mapping_kwargs.num_layers = 2
    c.G_opt_kwargs.lr = (0.002 if opts.cfg == 'stylegan2' else 0.0025) if opts.glr is None else opts.glr
    c.D_opt_kwargs.lr = opts.dlr
    c.metrics = opts.metrics
    c.total_kimg = opts.kimg
    c.kimg_per_tick = opts.tick
    c.image_snapshot_ticks = c.network_snapshot_ticks = opts.snap
    c.random_seed = c.training_set_kwargs.random_seed = opts.seed
    c.data_loader_kwargs.num_workers = opts.workers

    # Sanity checks.
    if c.batch_size % c.num_gpus != 0:
        raise ValueError('--batch must be a multiple of --gpus')
    if c.batch_size % (c.num_gpus * c.batch_gpu) != 0:
        raise ValueError('--batch must be a multiple of --gpus times --batch-gpu')
    if any(not metric_main.is_valid_metric(metric) for metric in c.metrics):
        raise ValueError('\n'.join(['--metrics can only contain the following values:'] + metric_main.list_valid_metrics()))

    # Base configuration.
    c.ema_kimg = c.batch_size * 10 / 32
    if opts.cfg == 'stylegan2':
        c.G_kwargs.class_name = 'pg_modules.networks_stylegan2.Generator'
        c.G_kwargs.fused_modconv_default = 'inference_only' # Speed up training by using regular convolutions instead of grouped convolutions.
        use_separable_discs = True

    elif opts.cfg == 'fastgan':
        c.G_kwargs = dnnlib.EasyDict(class_name='pg_modules.networks_fastgan.Generator', cond=opts.cond)
        c.G_opt_kwargs.lr = c.D_opt_kwargs.lr = 0.0002
        use_separable_discs = False

    # Restart.
    c.restart_every = opts.restart_every

    # Description string.
    desc = f'{opts.cfg:s}-{dataset_name:s}-gpus{c.num_gpus:d}-batch{c.batch_size:d}'
    if opts.desc is not None:
        desc += f'-{opts.desc}'

    # Projected and Multi-Scale Discriminators
    c.loss_kwargs = dnnlib.EasyDict(class_name='training.loss.ProjectedGANLoss')
    c.D_kwargs = dnnlib.EasyDict(
        class_name='pg_modules.discriminator.ProjectedDiscriminator',
        diffaug=True,
        interp224=(c.training_set_kwargs.resolution < 224),
        backbone_kwargs=dnnlib.EasyDict(),
    )

    c.D_kwargs.backbone_kwargs.cout = 64
    c.D_kwargs.backbone_kwargs.expand = True
    c.D_kwargs.backbone_kwargs.proj_type = 2
    c.D_kwargs.backbone_kwargs.num_discs = 4
    c.D_kwargs.backbone_kwargs.separable = use_separable_discs
    c.D_kwargs.backbone_kwargs.cond = opts.cond

    # Launch.
    launch_training(c=c, desc=desc, outdir=opts.outdir)

In [ ]:
!ls /kaggle/working/datasets

In [ ]:
!/kaggle/working/projected_gan/network-snapshot.zip

In [ ]:
import shutil
shutil.make_archive('network-snapshot', 'zip', '/kaggle/working/projected_gan/training-runs/00001-fastgan-img0-gpus1-batch64-')
#/kaggle/working/projected_gan/training-runs/00001-fastgan-img0-gpus1-batch64-/network-snapshot.pkl'

In [ ]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/projected_gan/network-snapshot.zip')

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working/projected_gan/','network-snapshot')

In [ ]:
# start training!

train(
    outdir='training-runs',
    cfg='fastgan',
    data='/kaggle/working/datasets/img0.zip',
    resume='/kaggle/working/projected_gan/training-runs/00001-fastgan-img0-gpus1-batch64-/network-snapshot.pkl',
    gpus=1,
    batch=64,
    cond=False,
    mirror=1,
    batch_gpu=8,
    cbase=32768,
    cmax=512,
    glr=None,
    dlr=0.002,
    desc='',
    metrics=[],
    kimg=500,
    tick=4,
    snap=1,
    seed=0,
    workers=0,
    restart_every=999999,
)

In [ ]:
!rm -rf /kaggle/working/projected_gan/network-snapshot.zip

In [ ]:
!ls  /kaggle/working/projected_gan/ 

In [ ]:
%%bash
python /kaggle/working/projected_gan/train.py --outdir=./training-runs/ --cfg=fastgan --data=/kaggle/working/datasets/img.zip \
  --gpus=8 --batch=64 --mirror=1 --snap=50 --batch-gpu=8 --kimg=10

In [ ]:
tf.keras.utils.plot_model(model_Inception, to_file='model.png', show_shapes=True, show_layer_names=True,show_dtype=True,dpi=120)

In [ ]:
checkpoint_cb =ModelCheckpoint("model_Inception.h5", save_best_only=True)
early_stopping_cb =EarlyStopping(patience=10, restore_best_weights=True)
model_Inception.compile(optimizer ='adam', loss='categorical_crossentropy', metrics=['accuracy'])
hist = model_Inception.fit(X_train,y_train, epochs=100, validation_data=(X_val,y_val), callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=(176,176,3),include_top=False,weights='imagenet')
base_model.trainable = False
model_Inception=keras.models.Sequential()
model_Inception.add(base_model)
model_Inception.add(keras.layers.Dropout(.5))
model_Inception.add(keras.layers.GlobalAveragePooling2D()) 
model_Inception.add(keras.layers.Flatten()) 
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dense(512,activation=tf.nn.relu))
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dropout(.5))
model_Inception.add(keras.layers.Dense(256,activation=tf.nn.relu))
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dropout(.5))
model_Inception.add(keras.layers.Dense(128,activation=tf.nn.relu))
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dropout(.5))
model_Inception.add(keras.layers.Dense(64,activation=tf.nn.relu))
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dropout(.5))
model_Inception.add(keras.layers.BatchNormalization())
model_Inception.add(keras.layers.Dense(4, activation=tf.nn.softmax))
model_Inception.summary()

<a id="11"></a>
# <p style="background-image: url(https://i.postimg.cc/K87ByXmr/stage5.jpg);font-family:camtasia;font-size:120%;color:white;text-align:center;border-radius:15px 50px; padding:7px"> Evaluate The Model  </p>
<a class="btn" href="#home">Tabel of Contents</a>

In [ ]:
hist_=pd.DataFrame(hist.history)
hist_

In [ ]:
plt.figure(figsize=(15,10))
plt.subplot(1,2,1)
plt.plot(hist_['loss'],label='Train_Loss')
plt.plot(hist_['val_loss'],label='Validation_Loss')
plt.title('Train_Loss & Validation_Loss',fontsize=20)
plt.legend()
plt.subplot(1,2,2)
plt.plot(hist_['accuracy'],label='Train_Accuracy')
plt.plot(hist_['val_accuracy'],label='Validation_Accuracy')
plt.title('Train_Accuracy & Validation_Accuracy',fontsize=20)
plt.legend()
plt.show()

In [ ]:
score, acc= model_Inception.evaluate(X_test,y_test)
print('Test Loss =', score)
print('Test Accuracy =', acc)

In [ ]:
predictions = model_Inception.predict(X_test)
y_pred = np.argmax(predictions,axis=1)
y_test_ = np.argmax(y_test,axis=1)
df = pd.DataFrame({'Actual': y_test_, 'Prediction': y_pred})
df

In [ ]:
plt.figure(figsize=(50,50))
for n,i in enumerate(np.random.randint(0,len(X_test),50)):
    plt.subplot(10,5,n+1)
    plt.imshow(X_test[i])
    plt.axis('off')
    plt.title(f'{class_num[y_test_[i]]} >>> {class_num[y_pred[i]]}',fontsize=15)

In [ ]:
CM = confusion_matrix(y_test_,y_pred)
CM_percent = CM.astype('float') / CM.sum(axis=1)[:, np.newaxis]
sns.heatmap(CM_percent,fmt='g',center = True,cbar=False,annot=True,cmap='Blues')
CM

In [ ]:
ClassificationReport = classification_report(y_test_,y_pred)
print('Classification Report is : ', ClassificationReport )